In [1]:
import pandas as pd

## 主成分分析原理实现

- 求相关系数矩阵R及其特征值向量化

In [ ]:
R = df2.corr()
eig_value, eig_vector = nlg.eig(R)

eig = pd.DataFrame()
eig['names'] = df2.columns
eig['eig_value'] = eig_value
eig.sort_values(by='eig_value',ascending=False,inplace=True)
eig

In [2]:
pd.read_clipboard()

,names,eig_value
0,total_sales,2.42759
3,gross_profit_margin,1.00306
2,gross_profit,0.51256
1,total_quantity,0.05679


- 确定公共因子个数，求因子载荷矩阵A

In [ ]:
for m in range(1,6):
    if eig['eig_value'][:m].sum()/eig['eig_value'].sum() >=0.8:  #使用前m个特征值的比重大于80%的标准
        print ('主成分个数为 %i'%m)
        break

主成分个数为 2

In [ ]:
A = np.zeros((4,2))
A[:,0] = sqrt(eig_value[0])*eig_vector[:,0]
A[:,1] = sqrt(eig_value[2])*eig_vector[:,2]
A = np.mat(A)
pd.DataFrame(A)

In [3]:
pd.read_clipboard()

,0,1
0,0.94563,-0.27936
1,0.78673,0.61718
2,0.95578,-0.23123
3,-0.03054,0.01206


> 可知因子0和因子1对信息的利用率占比超过八成（“二八定律”）

In [ ]:
h = np.zeros(4) #共同度
D = np.mat(np.eye(4))  #特殊因子方差
for i in range(4):
    a = A[i,:]*A[i,:].T
    h[i]=a[0,0]
    D[i,i] = 1-a[0,0]
print (h)
pd.DataFrame(D)

[0.97225402 0.99984236 0.9669766  0.00107832]

In [4]:
pd.read_clipboard()

,0,1,2,3
0,0.02775,0.00000,0.00000,0.00000
1,0.00000,0.00016,0.00000,0.00000
2,0.00000,0.00000,0.03302,0.00000
3,0.00000,0.00000,0.00000,0.99892


> 对载荷矩阵旋转变换后，主因子之间有相关性，斜交模型成立

In [ ]:
a = pd.DataFrame(A)
a.columns = ['factor1','factor2']
a.index =['x1','x2','x3','x4']
a

In [6]:
pd.read_clipboard()

,factor1,factor2
x1,0.94563,-0.27936
x2,0.78673,0.61718
x3,0.95578,-0.23123
x4,-0.03054,0.01206


> **销售额与毛利额权重占比在80%左右，其中销售额的约占3/5，并对整个销售指标体系施加了决定性影响**

## PCA直接实现的样例

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

- 使用pca降维为辅助列再计算cosine similarity

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1,whiten=False) #此处=0.9即为保持信息度90%，其结果与选择1维一样
pca.fit(df)
df['pca'] = pca.transform(df)
result = cosine_similarity(df)

In [ ]:
df_cosine_similarity = pd.DataFrame(result,index=df.index,columns=df.index)